In [1]:
## REMEMBER TO DELETE THIS LINE BEFORE UPLOADING
api_key = ...

In [2]:
import cohere  
co = cohere.Client(api_key)  
texts = [  
   'Hello from Cohere!'
]  
response = co.embed(texts=texts, model='embed-english-light-v2.0')  
embeddings = response.embeddings # All text embeddings 
print(len(embeddings[0])) # Print embeddings for the first text

1024


In [6]:
import pinecone      

pinecone.init(      
	api_key=...,      
	environment='us-west1-gcp'      
)      
index = pinecone.Index('pokemon-cards')

In [8]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [9]:
pinecone.describe_index("pokemon-cards")

IndexDescription(name='pokemon-cards', metric='cosine', replicas=1, dimension=1024.0, shards=1, pods=1, pod_type='p2.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [11]:
from datasets import load_dataset

In [15]:
pcds = load_dataset("TheFusion21/PokemonCards", split='train')
pcds

Dataset({
    features: ['id', 'image_url', 'caption', 'name', 'hp', 'set_name'],
    num_rows: 13139
})

In [20]:
pcds = pcds.map(lambda example: {"text" : f"Name: {example['name']}\nSet Name: {example['set_name']}\nDescription: {example['caption']}"})

In [21]:
from time import time, sleep

In [28]:
from torch.utils.data import DataLoader

In [29]:
dl = DataLoader(pcds, batch_size=64)

In [25]:
emb = co.embed(texts=texts, model='embed-english-light-v2.0')

In [10]:
pinecone_obj = {'id': None, 'values': None, 'metadata' : None}

In [34]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [40]:
for batch in dl:
    # print(batch)
    upsert_list = []
    texts = batch['text']
    response = co.embed(texts=texts, model="embed-english-light-v2.0")
    embs = response.embeddings
    # print(embs[0])

    for i in range(len(batch['id'])):
        pcd = pinecone_obj.copy()
        pcd['id'] = batch['id'][i]
        pcd['values'] = embs[i]
        pcd['metadata'] = {"img_url": batch['image_url'][i],
                           "name" : batch['name'][i], 
                           "description": batch['caption'][i], 
                           "hp": int(batch['hp'][i])}
        upsert_list.append(pcd)
    
    # print(upsert_list[0])
    upsert_resp = index.upsert(vectors=upsert_list)
    # sleep(1)
    # break

In [81]:
query = "sad"

response = co.embed(texts=[query], model="embed-english-light-v2.0")
query_emb = response.embeddings[0]
query_emb

[0.6044922,
 0.17944336,
 -1.4912109,
 -0.3088379,
 -1.2353516,
 1.2099609,
 1.2470703,
 1.8046875,
 -0.6035156,
 0.8720703,
 -0.45581055,
 2.265625,
 0.453125,
 0.671875,
 0.95947266,
 -0.06994629,
 -1.3730469,
 -0.83984375,
 0.3330078,
 -2.3945312,
 0.15356445,
 0.91015625,
 -0.42578125,
 -0.30126953,
 1.1308594,
 0.09820557,
 3.3945312,
 3.4082031,
 2.6757812,
 0.42822266,
 -2.9609375,
 0.92089844,
 0.42651367,
 -1.3457031,
 -0.4099121,
 -0.9428711,
 0.04736328,
 -1.1357422,
 0.057617188,
 2.7421875,
 0.87158203,
 0.55908203,
 1.0703125,
 -1.1835938,
 -1.7402344,
 1.5839844,
 -0.019897461,
 0.7597656,
 0.0769043,
 3.2929688,
 -2.3242188,
 1.3271484,
 -2.1796875,
 1.8515625,
 0.59521484,
 -1.8642578,
 0.7949219,
 1.4863281,
 -2.0839844,
 1.390625,
 0.3642578,
 0.46240234,
 -0.87060547,
 -0.74902344,
 -0.6411133,
 0.9223633,
 -0.8354492,
 0.8955078,
 0.9111328,
 -1.1035156,
 0.14428711,
 -1.3076172,
 0.43701172,
 -0.4099121,
 0.018554688,
 -1.3691406,
 0.9868164,
 1.4873047,
 -1.05859

In [82]:
query_response = index.query(query_emb, top_k=10, include_metadata=True)

In [83]:
query_response['matches'][0]

{'id': 'swshp-SWSH003',
 'metadata': {'description': 'A Basic Pokemon Card of type Water with the '
                             'title Sobble and 60 HP of rarity Promo from the '
                             'set SWSH Black Star Promos and the flavor text: '
                             'When scared, this Pokemon cries. Its tears pack '
                             'the chemical punch of 100 onions, and attackers '
                             "won't be able to resist weeping. It has the "
                             'attack Bind with the cost Water, Colorless, the '
                             'energy cost 2 and the damage of 20 with the '
                             'description: Flip a coin. If heads, your '
                             "opponent's Active Pokemon is now Paralyzed. It "
                             'has weakness against Lightning 2. ',
              'hp': 60.0,
              'img_url': 'https://images.pokemontcg.io/swshp/SWSH003_hires.png',
              'name': 

In [84]:
qurls = [x['metadata']['img_url'] for x in query_response['matches']]
qurls[0]

'https://images.pokemontcg.io/swshp/SWSH003_hires.png'

In [85]:
from IPython.display import Image, display
from IPython.core.display import HTML 

In [86]:
import ipyplot

In [87]:
# for url in qurls:
#     display(Image(url=url, width=200))

ipyplot.plot_images(qurls, img_width=200)